In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import tensorflow as tf
import numpy as np
import collections

/anaconda3/envs/flow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [3]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1600px;height:750px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [4]:
x0 = np.array([[1.], [1.]])  # 2x1 vector 

A = tf.constant(np.array([[-0.5, 0], [0, -0.6]]), dtype=tf.float32, name='A')  # system dynamics
B = tf.constant(np.array([[1.], [1.]]), dtype=tf.float32, name='B')

Q = tf.constant([[1.,0.], [0.,1.]], name='Q')  # weighting
R = tf.constant(1., name='R')

x = tf.constant(x0, dtype=tf.float32, name="state")

T = 10

In [5]:
####### Defining network #######
# input: state x
# output: control u

input_layer = tf.placeholder(tf.float32, (None,2), name='in_layer')
# fc1 = tf.layers.dense(inputs=input_layer, units=2,) #activation=tf.nn.sigmoid)
# fc2 = tf.layers.dense(inputs=fc1, units=2, activation=tf.nn.sigmoid)
u = tf.layers.dense(inputs=input_layer, units=1, name='u_out_layer', reuse=tf.AUTO_REUSE)

### LOSS FUNCTION ### 
loss = tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x), 
              tf.matmul(tf.transpose(u), tf.multiply(R, u)), name='loss')

# xs = tf.identity(x, name='xs')
# us = tf.constant(0, name='us')
xs = x
us = u

# cond = lambda i, x, l, xs, us: i < T

# def body(i, x, l, xs, us):
#     next_i = i+1
#     next_x = tf.add(tf.matmul(A, x), tf.multiply(u,B))
#     next_l = tf.add(l,
#                     tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x),
#                            tf.matmul(tf.transpose(u), tf.multiply(R, u))))
#     next_xs = tf.concat(xs, next_x)
#     next_us = tf.concat(us, u)
#     return (next_i, next_x, next_l, next_xs, next_us)

# i, xloss_f, traj_f = tf.while_loop(cond, body, 
#                                    loop_vars=[tf.constant(0), x, loss, xs, us],
#                                    shape_invariants=[tf.TensorShape([1,]), tf.TensorShape([2, 1]), 
#                                                      tf.TensorShape([1,]) , tf.TensorShape([2, None]), 
#                                                      tf.TensorShape([1, None])])
# train = tf.train.GradientDescentOptimizer(0.01).minimize(xloss_f.loss)

for i in range(T):
    # LQR loss 
#     x_term = tf.matmul(tf.matmul(tf.transpose(x), Q), x, name='x_term')
#     u_term = tf.matmul(tf.transpose(u), tf.multiply(R, u), name='u_term')
#     loss = tf.add(loss, tf.add(x_term, u_term), name='loss')  # accumulate loss
    
    # Dynamics: advancing the system dynamics
    Ax = tf.matmul(A, x, name='Ax'+str(i))
    Bu = tf.multiply(u, B, name='Bu'+str(i))  # tf.multiply because u is a scalar
    x = tf.add(Ax, Bu, name='state'+str(i))  # next state vector

    loss = tf.add(loss, tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x), tf.matmul(tf.transpose(u), tf.multiply(R, u))), name='loss'+str(i))  # accumulate loss    
    
    u = tf.layers.dense(inputs=tf.transpose(x), units=1, name='u_out_layer', reuse=True)
    
    xs = tf.concat([xs, x], 1)
    us = tf.concat([us, u], 1)
    
opt = tf.train.GradientDescentOptimizer(0.0001)
train = opt.minimize(loss)
grads_and_vars = opt.compute_gradients(loss)

In [8]:
# show_graph(tf.get_default_graph().as_graph_def())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(250):
#         sess.run(train)
        sess.run(train, feed_dict={input_layer : x0.T})
        if i % 50 == 0:
#             results = sess.run([xs, us, loss, grads_and_vars], feed_dict={input_layer : x0.T})
            results = sess.run([xs, us, loss], feed_dict={input_layer : x0.T})
            labels  = "xs us loss".split(' ')
            print('training iteration', i)
            for label,result in zip(*(labels,results)) :
                print(label)
                print(result)
                print('')
            for g, v in grads_and_vars:
                print('gradients')
                print(str(sess.run(g, feed_dict={input_layer : x0.T})) + " - " + v.name)
            print('----------------------')

training iteration 0
xs
[[ 1.         -0.7173682   0.49605566 -0.2926046   0.1731691  -0.06723218
   0.02857424  0.01676741 -0.01364223  0.03252974 -0.01430157]
 [ 1.         -0.8173682   0.62779254 -0.4212523   0.27961823 -0.14841858
   0.0840093  -0.01935105  0.00635211  0.02189736 -0.01117512]]

us
[[-0.21736817  0.13737158 -0.04457678  0.02686681  0.01935238 -0.00504185
   0.03105453 -0.00525853  0.02570863  0.0019633   0.0186675 ]]

loss
[[4.349453]]

gradients
[[-12.06689]
 [-13.08315]] - u_out_layer/kernel:0
gradients
[-4.0653143] - u_out_layer/bias:0
----------------------
training iteration 50
xs
[[ 1.00000000e+00 -6.09071851e-01  3.70836347e-01 -1.49988860e-01
   7.31519759e-02  2.10332200e-02 -1.04900915e-02  5.09902090e-02
  -1.15798619e-02  3.94421443e-02  3.19138542e-03]
 [ 1.00000000e+00 -7.09071875e-01  4.91743565e-01 -2.59616822e-01
   1.53927639e-01 -3.47473770e-02  2.08749454e-02  3.32201943e-02
  -6.01687469e-03  3.72623391e-02  5.55053353e-04]]

us
[[-1.0907187e-01

## compute optimal loss with true LQR ricatti equation formulation

In [9]:
import controlpy

In [10]:
if type(A) != np.ndarray:  # if tensors not already evaluated
    with tf.Session() as sess:
        A, B, Q, R = sess.run([A, B, Q, R])

K, P, eig = controlpy.synthesis.controller_lqr_discrete_time(A, B, Q, R)
x = x0
loss = 0
for i in range(T):
    u = (-K@x)
    loss += x.T@Q@x + u.T*R*u
    x = A@x + B@u
    print(x, loss, '\n-------------------------------')

[[-0.11926123]
 [-0.21926125]] [[2.14496201]] 
-------------------------------
[[-0.00746017]
 [ 0.06446597]] [[2.21176192]] 
-------------------------------
[[ 0.01648575]
 [-0.02592392]] [[2.21613615]] 
-------------------------------
[[-0.01116197]
 [ 0.01263526]] [[2.2170885]] 
-------------------------------
[[ 0.00649124]
 [-0.0066709 ]] [[2.21737356]] 
-------------------------------
[[-0.00362815]
 [ 0.00362001]] [[2.21746035]] 
-------------------------------
[[ 0.00200435]
 [-0.00198173]] [[2.21748665]] 
-------------------------------
[[-0.00110336]
 [ 0.00108785]] [[2.21749461]] 
-------------------------------
[[ 0.00060672]
 [-0.00059767]] [[2.21749701]] 
-------------------------------
[[-0.00033351]
 [ 0.00032845]] [[2.21749774]] 
-------------------------------
